In [1]:
import nltk
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
import pprint
import gensim
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import pandas as pd
import numpy as np
!pip install gensim

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/LovelyCheuk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('file2_lily.csv')
df = df.drop(columns='Unnamed: 0')
df = df[df['Description'].notnull()]

In [3]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [6]:
Description_list = []
for text in df['Description']:
    stemedText = preprocess(text)
    Description_list.append(stemedText) 

In [7]:
dictionary = corpora.Dictionary(Description_list)
corpus = [dictionary.doc2bow(text) for text in Description_list]

In [8]:
num_topics = 5
passes = 10
lda = LdaModel(corpus,
              id2word=dictionary,
              num_topics=num_topics,
              passes=passes)

In [9]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda.print_topics(num_words=8))

[   (   0,
        '0.006*"world" + 0.006*"famili" + 0.005*"life" + 0.005*"help" + '
        '0.004*"recip" + 0.004*"time" + 0.003*"home" + 0.003*"love"'),
    (   1,
        '0.010*"book" + 0.007*"stori" + 0.006*"world" + 0.006*"time" + '
        '0.005*"life" + 0.004*"american" + 0.004*"year" + 0.004*"know"'),
    (   2,
        '0.007*"book" + 0.006*"love" + 0.005*"life" + 0.005*"world" + '
        '0.004*"stori" + 0.004*"time" + 0.004*"littl" + 0.003*"read"'),
    (   3,
        '0.007*"life" + 0.006*"world" + 0.005*"time" + 0.005*"year" + '
        '0.004*"stori" + 0.004*"book" + 0.003*"live" + 0.003*"american"'),
    (   4,
        '0.006*"book" + 0.006*"world" + 0.005*"time" + 0.004*"know" + '
        '0.004*"busi" + 0.004*"live" + 0.003*"best" + 0.003*"famili"')]


In [10]:
from operator import itemgetter
lda.get_document_topics(corpus[0],minimum_probability=0.05,per_word_topics=False)
sorted(lda.get_document_topics(corpus[5],minimum_probability=0,per_word_topics=False),key=itemgetter(1),reverse=True)

[(4, 0.9919419),
 (1, 0.0020192824),
 (3, 0.0020172102),
 (2, 0.0020163741),
 (0, 0.0020052346)]

In [11]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/Users/LovelyCheuk/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
